In [ ]:
!pip install pystan==2.19.1.1
!pip install prophet


     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 184kB 23.5MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-cp37-none-any.whl size=6639618 sha256=fba96386587608a0a8c6da627eaa6d13e5dcf9718ae022c2c1b01973beb66981
  Stored in directory: /root/.cache/pip/wheels/92/1d/02/bc0a91bd3a84e060722075d2b0b68e875e37ec7dc49451d513
Successfully built prophet
ERROR: fbprophet 0.7.1 has requirement cmdstanpy==0.9.5, but you'll have cmdstanpy 0.9.68 which is incompatible.
  Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5


In [ ]:
import pandas as pd
import numpy as np 
from prophet import Prophet

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/xxiang27/sikkaAIHackathon/main/data_file.csv')
df.head()

,id,month,year,visits,no_of_appts,production
0,34,3,2014,1,960,76.55
1,34,4,2014,488,1089,106333.80
2,34,5,2014,831,993,192230.40
3,34,6,2014,753,884,163515.12
4,34,7,2014,721,797,160185.90


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27170 entries, 0 to 27169
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           27170 non-null  int64  
 1   month        27170 non-null  int64  
 2   year         27170 non-null  int64  
 3   visits       27170 non-null  int64  
 4   no_of_appts  27170 non-null  int64  
 5   production   27170 non-null  float64
dtypes: float64(1), int64(5)
memory usage: 1.2 MB


In [ ]:
df.describe()

,id,month,year,visits,no_of_appts,production
count,27170.000000,27170.000000,27170.000000,27170.000000,27170.000000,2.717000e+04
mean,141.936069,6.582149,2015.817372,567.629923,272.300184,1.810539e+05
std,81.682146,3.457194,3.018620,380.785944,380.348904,1.340010e+05
min,1.000000,1.000000,2010.000000,1.000000,1.000000,-1.592922e+05
25%,73.000000,4.000000,2013.000000,321.000000,18.000000,9.867377e+04
50%,143.000000,7.000000,2016.000000,479.000000,144.000000,1.498361e+05
75%,210.000000,10.000000,2018.000000,724.000000,403.000000,2.239591e+05
max,284.000000,12.000000,2020.000000,3229.000000,4147.000000,1.230910e+06


In [ ]:
def turntoDate(df):
  lst = []
  for i in range(len(df)):
    lst.append(pd.Timestamp(year = df.loc[i, 'year'], month = df.loc[i, 'month'], day=None))
  df['date'] = lst
  return df

In [ ]:
df['day'] = 1
dates = pd.to_datetime(df[['month', 'year','day']].copy())
dates.head()

0   2014-03-01
1   2014-04-01
2   2014-05-01
3   2014-06-01
4   2014-07-01
dtype: datetime64[ns]

In [ ]:
df_final = pd.concat([dates, df],axis=1)
df_final.columns.values[0] = 'date'
df_final.head()

,date,id,month,year,visits,no_of_appts,production,day
0,2014-03-01,34,3,2014,1,960,76.55,1
1,2014-04-01,34,4,2014,488,1089,106333.80,1
2,2014-05-01,34,5,2014,831,993,192230.40,1
3,2014-06-01,34,6,2014,753,884,163515.12,1
4,2014-07-01,34,7,2014,721,797,160185.90,1


In [ ]:
# separate dataframe into separate dataframes for each practice 

import math 

def separate(df):
  '''takes a dataframe returns a list of dataframes, 
  one for each unique value under the id column'''
  lst = []
  grouped = df.groupby(df.id)
  for i in range(1, 285):
    lst.append(grouped.get_group(i))
  return lst 

def train_test_split(lst, add_test):
  '''takes a list of dataframes and returns the time series train and test split based on the percent'''
  test_set = []
  for df in lst:
    df_add = df.tail(add_test).copy()
    test_set.append(df_add)
    df.drop(df.tail(add_test).index, inplace = True)
  return lst, test_set

def find_final(lst):
  months_lst = []
  years_lst = []
  weird_ids = []
  for df in lst: 
    month = df.tail(1).iloc[0,2]
    months_lst.append(month)
    year = df.tail(1).iloc[0,3]
    years_lst.append(year)
    if month != 12 or year != 2020:
      weird_ids.append(df.tail(1).iloc[0,1])
  return months_lst, years_lst, weird_ids




In [ ]:
df_final.tail(1).iloc[0,0]

Timestamp('2020-12-01 00:00:00')

In [ ]:
train_lst = separate(df_final)
months, years, weird = find_final(train_lst)

In [ ]:
df_final.head()

,date,id,month,year,visits,no_of_appts,production,day
0,2014-03-01,34,3,2014,1,960,76.55,1
1,2014-04-01,34,4,2014,488,1089,106333.80,1
2,2014-05-01,34,5,2014,831,993,192230.40,1
3,2014-06-01,34,6,2014,753,884,163515.12,1
4,2014-07-01,34,7,2014,721,797,160185.90,1


In [ ]:
df_train, df_test = train_test_split(train_lst, 4)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


In [ ]:
import itertools

In [ ]:
def dentistry_season(ds):
  date = pd.to_datetime(ds)
  # high traffic
  if date.month in [11,12,1,2]:
    return 3 
  # med traffic
  if date.month in [8, 10,7, 4]:
    return 2
  # low traffic
  if date.month in [3,5,6,9]:
    return 1
  

def make_predictions_regressors(df_train, name, best_params):
  best_params = []
  make_predictions = {}
  for df in df_train:
    X = df[['date', name]]
    id = df.iloc[0, 1]
    X.rename(columns={"date": "ds", name: "y"}, inplace = True)
    X['season'] = X['ds'].apply(dentistry_season)
    

    # TUNING
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
        'seasonality_mode': ['additive', 'multiplicative']
    }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(X)  # Fit model with given params
        # df_cv = cross_validation(m, cutoffs=cutoffs, horizon='30 days', parallel="processes")
        # df_cv = cross_validation(m, horizon='30 days', parallel="processes")
        df_cv = cross_validation(m, initial='730 days', period='60 days', horizon = '120 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    best_params.append(tuning_results)
    # tuning_results['rmse'] = rmses
    # print(tuning_results)
    # END OF TUNING

    future = m.make_future_dataframe(periods = 4, freq = 'MS')
    future['season'] = future['ds'].apply(dentistry_season)

    forecast = m.predict(future)
    pred = forecast.tail(4)['yhat'].to_numpy()
    pred1 = forecast.tail(4)['yhat_upper'].to_numpy()
    pred2 = forecast.tail(4)['yhat_lower'].to_numpy()
    pred3 = forecast.tail(4)['trend'].to_numpy()
    pred4 = forecast.tail(4)['trend_lower'].to_numpy()
    pred5 = forecast.tail(4)['trend_upper'].to_numpy()
    make_predictions[id] = [pred,pred1,pred2,pred3,pred4,pred5]
  return make_predictions

def make_predictions(df_train):
  make_predictions = {}
  for df in train_lst:
    X = df[['date', 'production']]
    id = df.iloc[0, 1]
    X.rename(columns={"date": "ds", "production": "y"}, inplace = True)
    
    # configure
    n_lag = 1
    n_seq = 4
    n_test = 1
    X = X.drop(columns = ['ds'])
    series = X
    scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)
    # fit model
    model = fit_lstm(train, 1, 4, 1, 1500, 1)
    # make forecasts
    forecasts = make_forecasts(model, 1, train, test, 1, 4)
    # inverse transform forecasts and test
    forecasts = inverse_transform(series, forecasts, scaler, n_test+2)
    
    m = Prophet()
    m.fit(X)
    future = m.make_future_dataframe(periods = 4, freq = 'MS')
    forecast = m.predict(future)
    pred = forecast.tail(4)['yhat'].to_numpy()


    make_predictions[id] = forecasts[0]
  return make_predictions 
    


In [ ]:
best_params = []

from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

predictions = make_predictions_regressors(df_train, 'production', best_params)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 10 forecasts with cutoffs between 2018-10-11 00:00:00 and 2020-04-03 00:00:00
INFO:prophet:Applying in parallel with <concur

In [ ]:
 def MAPE(y_true, y_pred):
	return np.mean(np.abs((y_true - y_pred)/y_true))*100

# need to make both of them dictionaries 
def score(predictions, actual):
  scores = []
  cate = [0,0,0,0]
  for i in range(1, 285):
    score = MAPE(actual[i], predictions[i])
    scores.append(score)
    if score <= 5:
      cate[0] += 1
    elif score > 5 and score <=10:
      cate[1] += 1
    elif score >10 and score <= 15:
      cate[2] += 1
    else:
      cate[3] += 1
  total_score = (cate[0]/284 * 2) + (cate[1]/284 * 1) + (cate[2]/284 * -1) + (cate[3]/284 * -2)
  return scores, total_score 

def prepare_test(df_test, name):
  actual = {}
  for df in df_test:
    id = df.iloc[0,1]
    actual[id] = df[name].to_numpy()
  return actual 

In [ ]:
actual = prepare_test(df_test)

In [ ]:
scores, total = score(predictions, actual)

In [ ]:
total

In [ ]:
plt.hist(scores)

In [ ]:
def prepare_train(df_train):
  train = {}
  for df in df_train:
    id = df.iloc[0,1]
    train[id] = df
  return train

def get_data_counts(train):
  counts = []
  for i in range(1, 285):
    df = train[i]
    counts.append(len(df))
  return counts


In [ ]:
counts = get_data_counts(prepare_train(df_train))


In [ ]:
plt.scatter(counts, scores)

In [ ]:
plt.hist(counts)

In [ ]:
np.mean(scores)